In [1]:
import pandas as pd

In [107]:
df = pd.read_csv('outFile.fasta_mean.csv')
df_origional

In [108]:
# Scripts to process BacTermFinder output 
#For Gene viewer at https://colab.research.google.com/drive/13aW6Kezl-XaPjJ9lY94YDFebeAqbf70f?usp=sharing
df.a = df['SampleName'].str.split('_').str # splitting

# Some namings
df['start'] =df.a[2].astype(int)
df['end'] = df.a[3].astype(int)
print('start and end done')
df['strand'] = df.a[4]
df['Accession/Start-Stop'] = df.a[1]
print("strand and chrom done")
df['name'] = 'bactermfinder'
df['score'] = df['probability_mean']

# Sorting columns for a bedfile
df = df[['Accession/Start-Stop', 'start', 'end', 'name', 'score', 'strand']]

# REMOVE NEGATIVE STRAND ROWS
df = df[df['strand'] != '-']

# Filtering based on the threhsold
test = df[df['score']>0.3]
# Saving to bed
test.to_csv('test_r_capsul.bed', index=False, sep='\t', header=False)

In [109]:
# First sort predictions by decreasing score WITHIN each Accession/Start-Stop
df_sorted = (df.groupby('Accession/Start-Stop', group_keys=False)
             .apply(lambda x: x.sort_values('score', ascending=False))
             .reset_index(drop=True))

In [110]:
# Create ordered list of tuples (guaranteed decreasing score order)
summary_df = df_sorted.groupby('Accession/Start-Stop').apply(
    lambda x: pd.Series({
        'terminator windows (start-stop), score': [(f"{row['start']}-{row['end']}", row['score']) 
                   for _, row in x.iterrows()],
        'top_score': x['score'].iloc[0],
    })
).reset_index()

In [111]:
# Add the missing rows (from previous step)
missing_ids = ['AAFX01046182.1/76-12', 'CBIH010000205.1/2901-2971']
missing_rows = []

for id_val in missing_ids:
    row = {'Accession/Start-Stop': id_val, 'terminator windows (start-stop), score': 'None', 'top_score': 0}
    missing_rows.append(row)

missing_df = pd.DataFrame(missing_rows)
summary_df = pd.concat([summary_df, missing_df], ignore_index=True)

# Re-sort by SampleName (alphabetical/natural order)
summary_df = summary_df.sort_values('Accession/Start-Stop').reset_index(drop=True)

In [114]:
# Export to Excel (without index)
summary_df.to_excel("BacTermFinder_Results_Fluoride.xlsx", index=False)

In [113]:
# Count rows with top_score over 0.3
count_over_0_3 = len(summary_df[summary_df['top_score'] > 0.3])
print(f"Riboswitches with top_score > 0.3: {count_over_0_3}/1901 ({count_over_0_3/1901:.1%})")